In [ ]:
# Importamos las librerias
import pandas as pd
import numpy as np
import seaborn as sns

# Cargamos el dataset 'tips' desde seaborn
data = sns.load_dataset('tips')

# Mostramos las primeras filas del dataset
data.head()

*_ANÁLISIS DE ESTADÍSTICAS DESCRIPTIVAS:_*

In [ ]:
#Informacion del dataset:
# Se muestra los tipos de datos, cantidad de valores no nulos y tamaño del dataset
data.info()

In [ ]:
#Resumen estadístico de variables numéricas:
# Nos proporciona estadísticas como media, desviación estándar, mínimo, máximo e información sobre los cuartiles.
data.describe()

In [ ]:
#Resumen de variables categóricas:
# Se proporciona estadísticas sobre datos categóricos (valores únicos y frecuencia).
data.describe(include="category")

In [ ]:
#Verificación de valores nulos por columna:
# Se detecta si hay datos faltantes en alguna columna.
data.isnull().sum()

In [ ]:
#Valores únicos en columnas categóricas:
# Se muestra los distintos valores que existen en cada columna categórica.
for col in data.select_dtypes(include=["category"]).columns:
    print(data[col].unique())

In [ ]:
#Distribución de la variable tip:
# Para mejor entendimiento de la columna <tips>, se procede a ver su distribución.
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.set_style("whitegrid")
sns.histplot(data["tip"], kde=True, bins=20, color="darkgoldenrod", edgecolor="black")
plt.title('Distribución de la columna tip', fontsize=16, fontweight="bold", color="peru")
plt.xlabel('Tip', fontsize=14, fontweight="bold")
plt.ylabel('Frecuencia', fontsize=14, fontweight="bold")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#Interacción entre variables
# Se puede crear una nueva variable que puedan tener una relación conjunta con la columna <tip>.
data['bill_per_person'] = round(data['total_bill'] / data['size'], 2)
data.head()

In [ ]:
#Matriz de correlación:
# Se analiza la relación entre las variables numéricas.
# Se selecciona solo columnas numéricas
columnas_num = data.select_dtypes(include=['float64', 'int64'])
columnas_num.corr()

In [ ]:
#Se hace una representación gráfica de esta misma matriz de correlación:
matriz_corr = columnas_num.corr()
plt.figure(figsize=(8,6))
sns.set_style("white")
sns.heatmap(matriz_corr, annot=True, cmap="rocket", fmt='.2f', linewidths=0.5,  linecolor='black')
plt.title('Matriz de Correlación entre Variables Numéricas', fontsize=16, fontweight='bold', color="indianred")
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.show()


*_LIMPIEZA DE DATOS:_*

1. Revisar valores nulos.
2. Detectar y tratar valores atípicos (outliers).
3. Revisar y convertir tipos de datos si es necesario.
4. Eliminar columnas irrelevantes.

Primero, verificamos si existen valores nulos en el dataset. Este paso se realizó anteriormente y el resultado fue negativo. Al no tener valores nulos, se sigue con la limpieza.

In [ ]:
#Detectar y tratar valores atípicos (outliers):
# Se puede detectar valores atípicos en las columnas numéricas utilizando diagramas de caja.
plt.figure(figsize=(10,6))
sns.set_style("white")
sns.boxplot(data=data[['total_bill', 'tip', 'size']], palette="magma")
plt.title("Detección de valores atípicos", fontsize=16, fontweight='bold', color="indianred")
plt.show()

Como hemos detectado valores atípicos extremos, podemos eliminarlos, considerando que son errores. Para eliminar outliers, podemos usar el rango intercuartílico (IQR) para filtrar datos del rango.

In [ ]:
# Se reconocen cuáles son el primer y tercer cuartil, para calcular el IQR
Q1 = data[['total_bill', 'tip', 'size']].quantile(0.25)
Q3 = data[['total_bill', 'tip', 'size']].quantile(0.75)
IQR = Q3 - Q1
# Se define los límites y se filtra los datos dentro del rango
data_limpio = data[~((data[['total_bill', 'tip', 'size']] < (Q1 - 1.5 * IQR)) |
                      (data[['total_bill', 'tip', 'size']] > (Q3 + 1.5 * IQR))).any(axis=1)]

print(f"Datos antes de limpiar outliers: {data.shape[0]} filas")
print(f"Datos después de limpiar outliers: {data_limpio.shape[0]} filas")

In [ ]:
#Revisar y convertir tipos de datos:
data_limpio.dtypes

Dado que las columnas ['sex', 'smoker', 'day', 'time'] se encuentran como category, no hace falta convertir ni modificar ningún tipo de variable.

In [ ]:
#Eliminar columnas irrelevantes, si hace falta
data_limpio.nunique()

Análisis de las columnas:

- total_bill y tip: Variables numéricas importantes para el modelo.
- sex, smoker, day, time, size: Tienen pocos valores únicos, lo que indica que son variables categóricas útiles para la predicción.

En este caso, no parece haber columnas irrelevantes, ya que todas contienen información potencialmente útil para el modelo.

*_ANÁLISIS EXPLORATORIO DE DATOS (EDA)_*

- EDA univariante: Para entender la distribución de cada variable.
- EDA bivariante: Para analizar la relación entre dos variables.
- EDA multivariante: Para explorar la relación entre múltiples variables simultáneamente.

Este paso sirve para comprender la distribución de los datos y así saber si las variables siguen una distribución normal o están sesgadas. También nos ayuda a identificar patrones y relaciones y detectar problemas.

In [ ]:
#EDA Univariante (Análisis de una variable a la vez)
# Variables numéricas
col_numeric = ['total_bill', 'tip', 'size', 'bill_per_person']

plt.figure(figsize=(15, 5))
for i, col in enumerate(col_numeric, 1):
    sns.set_style("white")
    plt.subplot(1, 4, i)
    sns.histplot(data_limpio[col], kde=True, color="palevioletred", edgecolor="black")
    plt.title(f'Distribución de {col}', fontsize=16, fontweight="bold", color="violet")
plt.tight_layout()
plt.show()

Con esto podemos entender mejor las variables numéricas. Por ejemplo, vemos que la cuenta, total_bill, suele estar en valores de 10-20 dólares. También vemos que la propina, tip, suele estar en valores de 2 o 3 dólares aproximadamente. Finalmente, se entiende que el size, número de personas en la mesa del restaurante, suele rondar a dos personas.

In [ ]:
# Variables categóricas
col_categoric = ['sex', 'smoker', 'day', 'time']

plt.figure(figsize=(15, 10))
for i, col in enumerate(col_categoric, 1):
    plt.subplot(2, 2, i)
    sns.set_style("white")
    sns.countplot(data=data_limpio, x=col,hue=col, palette="dark:salmon_r", edgecolor="black", legend=False)
    plt.title(f'Conteo de {col}', fontsize=16, fontweight="bold", color="crimson")
    plt.xlabel(col.capitalize(), fontsize=16, fontweight="bold")
    plt.ylabel('Frecuencia', fontsize=16, fontweight="bold")
plt.tight_layout()
plt.show()

Estos resultados nos muestra que los clientes más frecuentes suelen ser hombres. Además, este restaurante suele ser frecuentado por clientes no fumadores en los días de sábado y domingo y en horario de cena.

In [ ]:
#EDA Bivariante (Relación entre dos variables)
# Se realiza un análisis gráfico para examinar las relaciones entre las variables numéricas (total_bill, tip, size) y una de las variables categóricas de tu conjunto de datos.

for i, col in enumerate(col_categoric, 1):
    plt.figure(figsize=(10, 6))
    sns.set_style("white")
    g = sns.pairplot(data_limpio[['total_bill', 'tip', 'size', 'bill_per_person', col]], diag_kind='kde', palette='magma', hue=col)
    g.fig.suptitle(f'Relaciones entre variables numéricas según {col}', fontsize=16, fontweight="bold", color="mediumvioletred", y=1.02)
    plt.show()


In [ ]:
#Relación entre variables categóricas y la columna <tip> como se nos pide en la actividad (boxplots)
plt.figure(figsize=(15, 10))
for i, col in enumerate(col_categoric, 1):
    plt.subplot(2, 2, i)
    sns.boxplot(x=data_limpio[col], y=data_limpio['tip'], palette="flare", hue=data_limpio[col])
    plt.title(f'Propina según {col}', fontsize=16, fontweight="bold", color="rebeccapurple")
plt.tight_layout()
plt.show()

In [ ]:
#EDA Multivariante (Relación entre múltiples variables)
# Se visualiza la relación entre varias variables simultáneamente.
# Matriz de correlación para variables numéricas, a diferencia de la anterior matriz, esta se realizará con los datos limpiados, por ende se observa una diferencia en los resultados

nueva_corr_matrx = data_limpio[['total_bill', 'tip', 'size', 'bill_per_person']].corr()
plt.figure(figsize=(8,6))
sns.set_style("white")
sns.heatmap(nueva_corr_matrx, annot=True, cmap='YlOrBr', fmt='.2f', linewidths=0.5, linecolor='black')
plt.title('Matriz de Correlación entre Variables Numéricas', fontsize=16, fontweight='bold', color="salmon")
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)
plt.show()


In [ ]:
#Gráficos 3D (para variables numéricas)
# Aquí estamos visualizando las relaciones entre 'total_bill', 'tip', y 'size' en tres dimensiones

fig = plt.figure(figsize=(10, 8))
sns.set_style("white")
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_limpio['total_bill'], data_limpio['tip'], data_limpio['size'], c=data_limpio['size'], cmap='Spectral')
ax.set_xlabel('Total Bill', fontsize=10, fontweight='bold', color="slategrey")
ax.set_ylabel('Tip', fontsize=10, fontweight='bold', color="slategrey")
ax.set_zlabel('Size', fontsize=10, fontweight='bold', color="slategrey")
plt.show()

In [ ]:
#Análisis de Componentes Principales (PCA)
# El PCA es útil para reducir la dimensionalidad de los datos, especialmente cuando se tiene muchas variables. Permite proyectar los datos en un espacio de menor dimensión para observar la variabilidad en los datos.
# En este caso, se reduce las 3 variables originales a 2 componentes principales, lo que facilita la visualización y el análisis. También ayuda a ver cómo las tres variables interactúan y se agrupan, lo cual es difícil de hacer con más de dos variables.
# Se utiliza el color de los puntos para representar la variable size, lo que añade una dimensión extra de análisis

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Se normaliza Normalizar los datos antes de aplicar PCA
scaler = StandardScaler()
data_norm = scaler.fit_transform(data_limpio[['total_bill', 'tip', 'size']])

# Aplicando PCA
pca = PCA(n_components=2)
pca_components = pca.fit_transform(data_norm)

# Visualizando los dos primeros componentes principales
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=data_limpio['size'], cmap='magma')
plt.title('PCA: Primeros dos componentes principales', fontsize=10, fontweight='bold', color="slategrey")
plt.xlabel('Componente Principal 1', fontsize=10, fontweight='bold', color="slategrey")
plt.ylabel('Componente Principal 2', fontsize=10, fontweight='bold', color="slategrey")
plt.colorbar(label='Tamaño')
plt.show()

Al ver este gráfico, se puede ver que estas tres variables tienen relaciones lineales fuertes entre sí, y eso puede estar causando la alineación de los puntos en líneas.

En otras palabras, la relación entre total_bill, tip y size es aproximadamente lineal, por ende, los primeros componentes principales del PCA capturan esas relaciones lineales. En este caso, el PCA ha creado un espacio de componentes donde los datos se distribuyen a lo largo de tres líneas casi paralelas, lo que indica que las variables son altamente correlacionadas y que el PCA ha encontrado la estructura lineal subyacente.

Por ejemplo, la propina (tip) generalmente es una fracción del total de la factura (total_bill), y el tamaño (size) de la mesa puede afectar tanto al total de la factura como a la propina. Estas tres variables están relacionadas de manera proporcional (mesas más grandes tienden a tener una factura más alta y, por ende, una propina más alta), el PCA ha proyectado esos datos en una estructura lineal.

*_PREPARACIÓN DE DATOS PARA EL MODELADO_*

Resumen del proceso de preparación de datos:

+ Manejo de valores nulos → Imputación o eliminación.
+ Codificación de variables categóricas → One-Hot o Label Encoding.
+ Escalado de datos numéricos → Estandarización o normalización.
+ Eliminación de variables irrelevantes
+ División en conjuntos de entrenamiento y prueba.


In [ ]:
#Manejo de valores faltantes
# Como comprobamos anteriormente, este dataset no tiene valores nulos. Por consecuente, no hace falta hacer nada en este paso.
data_limpio.isna().sum()

In [ ]:
#Codificación de variables categóricas:
# Para el modelo que se planteara después, se ha escogido trabajar con One-Hot Encoding.
# Este sirve para transformar las categorías en columnas binarias (0 o 1). Es útil para variables categóricas como sex, smoker, day, time. No introduce relaciones espurias entre categorías aunque puede aumentar dimensionalidad si hay muchas categorías.

data_encoded = pd.get_dummies(data_limpio, columns=['sex', 'smoker', 'day', 'time'], drop_first=True)
data_encoded.head()

In [ ]:
#Conversión a 0 y 1:
# Para que los valores se muestren como 0 y 1 en lugar de False y True:

data_encoded = data_encoded.astype(int)
data_encoded.head()

In [ ]:
# Escalado de datos numéricos:
# Teniendo en cuenta que vamos a realizar un modelo de regresión, se usará Estandarización (StandardScaler).
# Este convierte los datos para que tengan media = 0 y desviación estándar = 1. Es el más indicado cuando los datos tienen una distribución normal o desconocida. También sirve cuando las variables tienen diferentes escalas (como total_bill y size).

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_encoded[['total_bill', 'tip', 'size']] = scaler.fit_transform(data_encoded[['total_bill', 'tip', 'size']])
data_encoded.head()

El siguiente paso consiste en la eliminación de variables irrelevantes. Anteriormente, he estudiado esto en la limpieza de datos, y llegué a la conclusión que todas las columnas aportan información relevante.

In [ ]:
#División en conjuntos de entrenamiento y prueba
# Para dividir los datos, se usará la función train_test_split, que permite separar el dataset en dos partes:
# 1. Conjunto de entrenamiento (train set): Se utiliza para entrenar el modelo. Suele representar el 70-80% del total de los datos.
# 2. Conjunto de prueba (test set): Se utiliza para evaluar el rendimiento del modelo. Suele representar el 20-30% del total de los datos.

from sklearn.model_selection import train_test_split

# Se define la columna <tip> como y las variables para predecir como X
X = data_encoded.drop(columns=['tip'])  # Todas menos la columna <tip>
y = data_encoded['tip']  # Solo la columna <tip>

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Se muestra la forma de los conjuntos para verificar
print("Shape de X_train:", X_train.shape)
print("Shape de X_test:", X_test.shape)
print("Shape de y_train:", y_train.shape)
print("Shape de y_test:", y_test.shape)



*_MODELADO_*

En mi caso, he decidido usar tanto la regresión lineal simple, aplicando la optimización Ridge y arboles usando regresión. Al final se comparará el rendimiento de todos y se tomará una decision sobre cuál será mejor emplear.

Pasos para el modelado de regresión

- Importar librerías necesarias.
- Optimización de hiperparámetros
- Crear y entrenar el modelo de regresión lineal.
- Realizar predicciones con el conjunto de prueba.
- Evaluar el modelo usando métricas adecuadas.


In [ ]:
#La optimización de hiperparámetros busca optimizar estos para encontrar la mejor configuración para el modelo antes de entrenarlo completamente con los datos de entrenamiento.
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Se define el modelo y se especifica los hiperparámetros clave.
ridge = Ridge()
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

# Se configura la búsqueda de hiperparámetros usando la técnica de GridSearchCV.
# GridSearchCV divide automáticamente los datos de entrenamiento en subconjuntos (validación cruzada) para evaluar cada combinación de hiperparámetros. Además, selecciona la mejor combinación de tal manera que maximize el desempeño de este.
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train) # Se entrena el modelo

# Se estudia cuál es el mejor modelo Ridge con hiperparámetros óptimos
best_ridge = grid_search.best_estimator_


Interpretación de los resultados:

- MAE: Indica el promedio de error en unidades monetarias (por ejemplo, dólares).
- MSE/RMSE: Cuanto más bajos, mejor será la predicción del modelo.
- R^2:
   - Si R^2 ≈ 1, el modelo se ajusta bien a los datos.
   - Si R^2 ≈ 0, el modelo no explica la variabilidad de los datos.
   - Si R^2 es negativo, el modelo es peor que una predicción basada en la media.

In [ ]:
# Evaluamos Ridge Regression en test
y_pred_ridge = best_ridge.predict(X_test)

print("Resultados Ridge Regression:")
print("Mejor alpha:", grid_search.best_params_['alpha'])
print("MAE:", mean_absolute_error(y_test, y_pred_ridge))
print("MSE:", mean_squared_error(y_test, y_pred_ridge))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_ridge)))
print("R²:", r2_score(y_test, y_pred_ridge))

In [ ]:
from sklearn.linear_model import LinearRegression

# Se crea el modelo de regresión lineal
model = LinearRegression()

# Se entrena el modelo con los datos de entrenamiento creados anteriormente
model.fit(X_train, y_train)

# Se realiza las predicciones en con los datos de validación creados anteriormente
y_pred_model = model.predict(X_test)

In [ ]:
 #Se evalua el rendimiento del modelo
mae = mean_absolute_error(y_test, y_pred_model)
mse = mean_squared_error(y_test, y_pred_model)
rmse = mean_squared_error(y_test, y_pred_model)
r2 = r2_score(y_test, y_pred_model)

# Se muestra los errores del modelo entrenado y testeado
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R^2: {r2:.2f}")

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Se crea el modelo de árbol de decisión
tree = DecisionTreeRegressor(max_depth=5, random_state=42)

# Se entrena el modelo con los datos de entrenamiento
tree.fit(X_train, y_train)

# Se realiza las predicciones con los datos de validación
y_pred_tree = tree.predict(X_test)


In [ ]:
# Se evalua el rendimiento del modelo
mae_tree = mean_absolute_error(y_test, y_pred_tree)
mse_tree = mean_squared_error(y_test, y_pred_tree)
rmse_tree = np.sqrt(mse)
r2_tree = r2_score(y_test, y_pred_tree)

print("Resultados Árbol de Regresión:")
print(f"MAE: {mae_tree:.4f}")
print(f"MSE: {mse_tree:.4f}")
print(f"RMSE: {rmse_tree:.4f}")
print(f"R²: {r2_tree:.4f}")

In [ ]:
#Validación cruzada (Cross-Validation)
# Se puede ver si el modelo está sobreajustado, realizando una validación cruzada:

from sklearn.model_selection import cross_val_score

scores_ridge = cross_val_score(ridge, X, y, cv=5, scoring='r2')
print("R^2 promedio en validación cruzada usando Ridge:", scores_ridge.mean())

scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print("R^2 promedio en validación cruzada sin usar Ridge:", scores.mean())

scores_tree = cross_val_score(tree, X, y, cv=5, scoring='r2')
print("R^2 promedio en validación cruzada usando arboles:", scores.mean())


In [ ]:
#Interpretación de resultados
# Coeficientes de la regresión: Muestra qué variables tienen mayor influencia.

print("Coeficientes:", model.coef_)
print("Intercepto:", model.intercept_)



In [ ]:
#Visualización de los resultados usando Ridge: Se compara los valores reales vs. predichos en un gráfico.
sns.set_style("white")
sns.scatterplot(x=y_test, y=y_pred_ridge, alpha=0.5, color="chocolate")
plt.xlabel("Valores Reales", fontsize=16, fontweight="bold", color="darkred")
plt.ylabel("Valores Predichos", fontsize=16, fontweight="bold", color="darkred")
plt.title("Valores Reales vs. Predichos", fontsize=16, fontweight="bold", color="darkred")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.show()


In [ ]:
# Visualización de los resultados sin usar Ridge: Se compara los valores reales vs. predichos en un gráfico.
sns.set_style("white")
sns.scatterplot(x=y_test, y=y_pred_model, alpha=0.5, color="darkorchid")
plt.xlabel("Valores Reales", fontsize=16, fontweight="bold", color="rebeccapurple")
plt.ylabel("Valores Predichos", fontsize=16, fontweight="bold", color="rebeccapurple")
plt.title("Valores Reales vs. Predichos", fontsize=16, fontweight="bold", color="rebeccapurple")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.show()

In [ ]:
# Visualización de resultados usando los árboles de decision con regresión: Se compara los valores reales vs. predichos en un gráfico.
sns.set_style("white")
sns.scatterplot(x=y_test, y=y_pred_tree, alpha=0.7, color="goldenrod")
plt.xlabel("Valores Reales", fontsize=16, fontweight="bold", color="orangered")
plt.ylabel("Valores Predichos", fontsize=16, fontweight="bold", color="orangered")
plt.title("Árbol de Decisión: Valores Reales vs Predichos", fontsize=16, fontweight="bold", color="orangered")
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
plt.show()

Resultados del Modelo sin Ridge:
- MAE (Error Absoluto Medio): 0.59, lo que indica cuán lejos están de las predicciones de los valores reales.
- MSE (Error Cuadrático Medio): 0.55. El MSE es más bajo que el MAE, lo que sugiere que los errores más grandes afectan más el rendimiento del modelo.
- RMSE (Raíz del Error Cuadrático Medio): 0.55, lo que refleja una dispersión moderada en las predicciones.
- R^2 (Coeficiente de Determinación): 0.39. Indica que el 39% de la variabilidad de la variable (tip) se puede explicar mediante las variables predictoras. Aunque es una buena base, se puede mejorar el modelo.
- R^2 Promedio en Validación Cruzada: 0.138. Un valor bajo sugiere: 1. el modelo puede estar sobreajustado, 2. los datos no son suficientemente representativos de todos los posibles patrones.

Resultados del Modelo con Ridge:
- MAE (Error Absoluto Medio): 0.60. Aumenta ligeramente en el modelo con Ridge, lo que indica que este podría estar penalizando más los coeficientes grandes.
- MSE (Error Cuadrático Medio): 0.58. También es un poco más alto, lo que refuerza la idea de que este afecta el rendimiento.
- RMSE (Raíz del Error Cuadrático Medio): 0.76. Es más alto que en el modelo sin Ridge. Esto significa que, ciertamente, el Ridge ayuda a reducir el sobreajuste, pero suprime demasiado los coeficientes, afectando la precisión de las predicciones.
- R^2 (Coeficiente de Determinación): 0.36, es algo inferior al modelo sin Ridge, lo que indica que se ha reducido ligeramente la capacidad del modelo para ajustar los datos.
- R^2 Promedio en Validación Cruzada: 0.15, ligeramente mejor que el modelo sin Ridge, otro indicativo que explica como ha ayudado a reducir el sobreajuste.

Resultados del Modelo con Árboles de Decisión:
- MAE (Error Absoluto Medio): 0.6081, un poco mayor que la regresión lineal (0.5900) y Ridge (0.5962), el error del árbol es ligeramente mayor que el de los modelos lineales.
- MSE (Error Cuadrático Medio): 0.5747, es un poco mayor que la regresión lineal (0.5500) pero menor que Ridge (0.5836). Muestra que el modelo del árbol tiene un error mayor, pero sigue siendo razonable.
- RMSE (Raíz del Error Cuadrático Medio): Similar al MSE, el árbol tiene un RMSE de 0.7581, mayor que los modelos lineales (0.7420), lo que sugiere un ligero empeoramiento en la precisión.
- R² (Coeficiente de Determinación): 0.3699, menor que la regresión lineal (0.3900), lo que indica que explica un poco menos la variabilidad de los datos en comparación con los modelos lineales.
- R^2 Promedio en Validación Cruzada: 0.138, el mismo valor que el modelo de regresión sin Ridge.

Coeficientes de la Regresión Sin Ridge:
- Los coeficientes obtenidos son más grandes en comparación con el otro, lo que indica que el modelo sin Ridge está "ajustando más" los datos, pero puede haber un sobreajuste. Por ejemplo: Coeficiente de sex_Female = -0.179: A medida que una persona es más propensa a ser mujer, la propina disminuye en ese valor.

CONCLUSION:
Valorando la capacidad de generalización y no priorizando un modelo que no se ajuste demasiado a los datos de entrenamiento, Ridge Regression es la mejor opción.
Sin embargo, si se prefiere un modelo que explote todo el poder predictivo de las características, es mejor optar por el modelo sin Ridge.
Finalmente, El modelo de árbol de decisión no supera a la regresión lineal en términos de precisión. Tiene un rendimiento ligeramente inferior y el error es un poco mayor, lo que sugiere que el modelo de regresión lineal es más adecuado para estos datos.
